# Bayesian cutoff analysis for Triton binding energy (LO/NLO/N2LO)

This notebook:
1. Parses TRITON output files grouped by chiral order (LO, NLO, N2LO) and cutoff values \(R\).
2. Estimates EFT uncertainty at each order from cutoff spread.
3. Computes discrete Bayesian posterior over \(R\) with a Gaussian likelihood.
4. Performs Bayesian Model Averaging (BMA) across cutoffs.
5. Produces plots and exports tables for the thesis.


In [1]:
import re
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Path to the folder that contains: cutoff_LO/, cutof_NLO/, cutoff_N2LO/
BASE_DIR = Path(r"results_cutoff")  

# Experimental value 
E_EXP = -8.482 #MeV

# Experimental uncertainty. Usually negligible vs EFT/systematics.
SIGMA_EXP = 0.01 #MeV

# Folder name mapping 
ORDER_MAP = {
    "cutoff_lo": "LO",
    "cutoff_nlo": "NLO",
    "cutoff_n2lo": "N2LO",
}

print("BASE_DIR:", BASE_DIR.resolve())

BASE_DIR: /mnt/c/Users/silvi/Desktop/TESI3/results_cutoff


In [3]:
def infer_order_from_path(p: Path) -> str:
    parts = [x.lower() for x in p.parts]
    for part in reversed(parts):
        if part in ORDER_MAP:
            return ORDER_MAP[part]
    raise ValueError(f"Cannot infer order from path: {p}")


def infer_R_from_filename(name: str) -> float:
    """
    Robust inference of R from filename.
    Supports:
      - R08 -> 0.8
      - R10 -> 1.0
      - R0.8 -> 0.8
      - any '0.8' substring
    """
    s = name.lower()

    # R08 / R10 / R12 pattern
    m = re.search(r"r(\d{2})", s)
    if m:
        two = int(m.group(1))
        return two / 10.0

    # R0.8 / R1.2 pattern
    m = re.search(r"r(\d+\.\d+)", s)
    if m:
        return float(m.group(1))

    # plain 0.8 pattern
    m = re.search(r"(\d+\.\d+)", s)
    if m:
        return float(m.group(1))

    raise ValueError(f"Cannot infer R from filename: {name}")


def read_triton_energy(file_path: Path) -> float:
    """
    Extract triton binding energy from a .dat file
    """
    text = file_path.read_text(errors="ignore")
    lines = text.splitlines()

    # float regex supports Fortran D exponents too
    float_re = r"[-+]?\d*\.\d+(?:[eEdD][-+]?\d+)?|[-+]?\d+(?:[eEdD][-+]?\d+)?"

    # targeted line search
    for line in lines[::-1]:
        low = line.lower()
        if ("triton" in low) or ("3h" in low) or ("binding" in low):
            nums = re.findall(float_re, line)
            if nums:
                return float(nums[-1].replace("D", "E").replace("d", "E"))

    # fallback: last number in file
    nums = re.findall(float_re, text)
    if not nums:
        raise ValueError(f"No numeric values found in {file_path}")
    return float(nums[-1].replace("D", "E").replace("d", "E"))

In [ ]:
rows = []
for fp in BASE_DIR.rglob("*.dat"):
    order = infer_order_from_path(fp)
    R = infer_R_from_filename(fp.name)
    E = read_triton_energy(fp)
    rows.append({"order": order, "R": R, "E_triton (MeV)": E, "file": str(fp)})

df = pd.DataFrame(rows)
if df.empty:
    raise RuntimeError(f"No .dat files found under {BASE_DIR.resolve()}")

df = df.sort_values(["order", "R"]).reset_index(drop=True)

df[["order", "R", "E_triton (MeV)"]]

KeyError: 'R'

In [ ]:
# Check we have 5 cutoffs per order (expected)
counts = df.groupby("order")["R"].count()
display(counts)

# Check unique R values
display(df.groupby("order")["R"].unique())

In [ ]:
eft = (
    df.groupby("order")["E_triton"]
      .std(ddof=1)
      .reset_index()
      .rename(columns={"E_triton": "sigma_eft"})
)

df = df.merge(eft, on="order", how="left")
df["sigma_tot"] = np.sqrt(SIGMA_EXP**2 + df["sigma_eft"]**2)

eft

In [ ]:
df["logL"] = -0.5 * (df["E_triton"] - E_EXP)**2 / (df["sigma_tot"]**2)
df["posterior_R"] = 0.0

for order in df["order"].unique():
    m = df["order"] == order
    logL = df.loc[m, "logL"].to_numpy()

    # uniform prior over available cutoffs
    w = np.exp(logL - logL.max())
    df.loc[m, "posterior_R"] = w / w.sum()

df[["order", "R", "E_triton", "sigma_eft", "posterior_R"]]

In [ ]:
bma_rows = []
for order in df["order"].unique():
    sub = df[df["order"] == order]
    w = sub["posterior_R"].to_numpy()
    E = sub["E_triton"].to_numpy()

    mean = float(np.sum(w * E))
    var = float(np.sum(w * (E - mean)**2))
    sd = float(np.sqrt(var))

    bma_rows.append({"order": order, "E_mean": mean, "E_sd": sd})

bma = pd.DataFrame(bma_rows)

# Force order ordering LO, NLO, N2LO
order_cat = pd.Categorical(bma["order"], categories=["LO", "NLO", "N2LO"], ordered=True)
bma = bma.assign(order=order_cat).sort_values("order").reset_index(drop=True)

bma

In [ ]:
plt.figure(figsize=(7,4))
plt.errorbar(bma["order"], bma["E_mean"], yerr=bma["E_sd"], fmt="o", capsize=5)
plt.axhline(E_EXP, linestyle="--")
plt.ylabel("Triton binding energy [MeV]")
plt.xlabel("Chiral order")
plt.title("Bayesian model averaging over cutoff")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(7,4))

for order in ["LO", "NLO", "N2LO"]:
    sub = df[df["order"] == order].sort_values("R")
    plt.plot(sub["R"], sub["posterior_R"], marker="o", label=order)

plt.xlabel("Cutoff R")
plt.ylabel("Posterior probability p(R | data)")
plt.title("Discrete posterior over cutoff")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
out_dir = Path("analysis_outputs")
out_dir.mkdir(exist_ok=True)

df.to_csv(out_dir / "parsed_triton_results.csv", index=False)
bma.to_csv(out_dir / "bma_results.csv", index=False)
eft.to_csv(out_dir / "eft_sigma_from_cutoff_spread.csv", index=False)

# Save figures
plt.figure(figsize=(7,4))
plt.errorbar(bma["order"], bma["E_mean"], yerr=bma["E_sd"], fmt="o", capsize=5)
plt.axhline(E_EXP, linestyle="--")
plt.ylabel("Triton binding energy [MeV]")
plt.xlabel("Chiral order")
plt.title("Bayesian model averaging over cutoff")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(out_dir / "bma_over_orders.png", dpi=200)
plt.close()

plt.figure(figsize=(7,4))
for order in ["LO", "NLO", "N2LO"]:
    sub = df[df["order"] == order].sort_values("R")
    plt.plot(sub["R"], sub["posterior_R"], marker="o", label=order)
plt.xlabel("Cutoff R")
plt.ylabel("Posterior probability p(R | data)")
plt.title("Discrete posterior over cutoff")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(out_dir / "posterior_over_R.png", dpi=200)
plt.close()

print("Saved outputs in:", out_dir.resolve())